In [3]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 1000000 

%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (14,4)

mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 8]

In [4]:
#읍면동 리스트
sample = pd.read_csv('data/2.세종시_표제부.csv')
sample['읍면동'] = sample['대지위치'].str.replace('세종특별자치시 ','')
sample['읍면동'] = sample['읍면동'].str.replace('조치원읍 ','조치원읍')
sample[['1', '2','3']] = sample['읍면동'].str.split(' ', n=2, expand=True)
sample['2'] = sample['2'].str.replace('조치원읍','조치원읍 ')
list_d = list(set(sample['2']))
data_d = pd.DataFrame(list_d,columns={'읍면동'})
#조치원읍/동/면 분리
list_myn = []
list_dong = []
for i in list_d :
  if '조치원읍 ' in i :
    pass;
  elif '동' in i :
    list_dong.append(i)
  else :
    list_myn.append(i)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
def make_rentfee_df(a,b,c) :
  df = pd.read_csv(a)
  df['보증금(만원)'] = df['보증금(만원)'].str.replace(',','')
  df = df.astype({'보증금(만원)' : float, '계약년월' : 'str'})
  df['시군구'] = df['시군구'].str.replace('세종특별자치시 ','')
  df.replace({'전세' : 'yr', '월세' : 'mth'},inplace=True)
  
  df_j = make_rentfee_data(df,b,'yr')
  df_m = make_rentfee_data(df,b,'mth')
  
  d = pd.merge(c,df_j, how='outer',on='읍면동')
  f = pd.merge(d,df_m, how='outer',on='읍면동')

  return f

def make_rentfee_data(i, b, c) :

  df = i[i['전월세구분'] == c]
  df[['읍면동', '2']] = df['시군구'].str.split(' ', n=1, expand=True)

  df = df.groupby(['읍면동']).agg({'계약일' : 'count', '보증금(만원)':'mean'})
  x = b+'_'+c+'_cnt'
  y = b+'_'+c+'_price'
  
  df.columns = [x,y]

  return df

In [9]:
ofi = make_rentfee_df('data/10.세종시_오피스텔(전월세)_실거래가.csv','ofi',data_d)
apt = make_rentfee_df('data/7.세종시_아파트(전월세)_실거래가.csv','apt',data_d)
ali = make_rentfee_df('data/8.세종시_연립다세대(전월세)_실거래가.csv','ali',data_d)
# 단독주택 제외 >> 대지단위(명확하게 비교하기가 어려움)
summary = pd.merge(ofi, apt, how='outer', on='읍면동')
summary = pd.merge(summary, ali, how='outer', on='읍면동')

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [145]:
def make_summary_by_location(a,b,c) :
    a = summary['읍면동'].str.contains(b)
    a = summary[a]
    a = a.reset_index(drop=True)
    #'전동면'만 제외
    if b == '동' :
        a = a.loc[a['읍면동'] != "전동면"]
        a = a.loc[a['읍면동'] != "연동면"]
        a = a.reset_index(drop=True)
    else :
        a = summary[summary['읍면동'] == '전동면']
        a = summary[summary['읍면동'] == '연동면']

    ex = pd.DataFrame()

    #market divieded
    for i in a['읍면동'] :
        df = market[market['행정동명']==i]
        df = df.groupby('상권업종중분류코드').agg({'행정동명':'count','상권업종중분류명' : 'first','상권업종대분류코드' :'first','상권업종대분류명' : 'first'})
        total = df['행정동명'].sum()
        
        df= df.transpose()
        df['읍면동'] = i
        df['total'] = total

        list_x = df.iloc[0]

        ex = ex.append(pd.Series(list_x), ignore_index=True)
        
    x = pd.merge(a,ex, how='outer', on='읍면동')
    x.head()
    
    return x

In [291]:
dong = make_summary_by_location(dong, '동','1')
dong_final = dong[dong['total'] != 0]
dong_final = dong_final.sort_index(axis=1)
dong[dong['total'] == 0]['읍면동']

0     나성동
1     해밀동
3     어진동
4     집현동
8     반곡동
10    합강동
12    산울동
14    다정동
15    가람동
Name: 읍면동, dtype: object

In [306]:
code = pd.read_csv('data/15.세종시_상권정보_업종코드.csv')
list_main = list(set(code['대분류코드']))
list_middle = list(set(code['중분류코드']))
list_middle = sorted(list_middle)
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대분류코드   837 non-null    object
 1   대분류명    837 non-null    object
 2   중분류코드   837 non-null    object
 3   중분류명    837 non-null    object
 4   소분류코드   837 non-null    object
 5   소분류명    837 non-null    object
dtypes: object(6)
memory usage: 39.4+ KB


In [320]:
for i in list_main :
    df = dong_final.iloc[:,74:]

    list_y = []
    list_x = [middle for middle in list_middle if i in middle]
    x = list_x[-1]

    corr = dong_final.loc[ :,list_x[0]:list_x[-1]]
    corr['읍면동'] = dong_final['읍면동']
    corr = pd.merge(df,corr,on='읍면동')

    print(i, list(set(code[code['대분류코드']==i]['대분류명'])))

    corr = corr.corr(method = 'pearson')
    display(corr)
    #df_heatmap = sns.heatmap(corr)
    #plt.savefig('{}'.format(i)+'.png')


D ['소매']


,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,ofi_mth_cnt,ofi_mth_price,...,D14,D15,D16,D17,D18,D20,D21,D23,D25,D26
ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,NaN,NaN,...,0.618961,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN
ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,NaN,NaN,...,-0.574980,NaN,NaN,NaN,-1.000000,NaN,NaN,1.000000,NaN,NaN
ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,...,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN
ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,NaN,NaN,...,0.699322,NaN,0.238953,NaN,0.499329,0.399090,0.643725,0.818745,0.956392,0.757304
apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,NaN,NaN,...,0.092900,NaN,0.102384,NaN,0.276131,0.359171,-0.599550,0.631848,-0.778265,0.372754
apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,NaN,NaN,...,0.381440,NaN,-0.092779,NaN,0.239528,0.131526,0.390195,0.336599,0.747750,0.250243
apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,NaN,NaN,...,0.601128,NaN,0.767889,NaN,0.732672,0.537240,0.823513,-0.828130,0.817813,0.059481
ofi_mth_cnt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ofi_mth_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Q ['음식']


,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,ofi_mth_cnt,ofi_mth_price,...,Q05,Q06,Q07,Q08,Q09,Q10,Q12,Q13,Q14,Q15
ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,NaN,NaN,...,0.959618,0.968372,0.608361,0.603902,0.720775,-1.000000,0.726267,NaN,NaN,-0.524781
ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,NaN,NaN,...,-0.942757,-0.953240,-0.563944,-0.559306,-0.681697,-1.000000,-0.687497,NaN,NaN,0.477334
ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,1.000000
ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,1.000000,NaN,NaN,-1.000000
apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,NaN,NaN,...,0.200473,0.532978,0.189587,0.498155,0.464094,-0.047125,0.497464,NaN,0.396784,-0.086794
apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,NaN,NaN,...,0.296361,-0.130152,0.254892,0.003029,-0.274090,-0.200528,-0.153926,NaN,0.939171,0.595906
apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,NaN,NaN,...,0.137199,0.531458,-0.189514,0.470578,0.544479,-0.060299,0.482028,NaN,0.065419,-0.568177
apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,NaN,NaN,...,0.845555,0.785699,0.583290,0.570950,0.566101,0.875138,0.597551,NaN,0.245011,0.151998
ofi_mth_cnt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ofi_mth_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


P ['스포츠']


...

,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,ofi_mth_cnt,ofi_mth_price,...,R02,R03,R04,R05,R06,R07,R08,R09,R10,R13
ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,NaN,NaN,...,NaN,NaN,0.028855,-0.759251,NaN,0.851237,0.524781,0.524781,NaN,1.000000
ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,NaN,NaN,...,NaN,NaN,0.025977,0.793781,NaN,-0.878722,-0.477334,-0.477334,NaN,1.000000
ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,...,NaN,NaN,-1.000000,-1.000000,NaN,1.000000,-1.000000,-1.000000,NaN,-1.000000
ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,...,NaN,NaN,1.000000,1.000000,NaN,-1.000000,1.000000,1.000000,NaN,1.000000
apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,NaN,NaN,...,NaN,NaN,0.270532,0.007576,NaN,0.183837,0.585301,0.510357,NaN,0.711627
apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,NaN,NaN,...,NaN,NaN,0.405425,0.154397,NaN,0.181769,0.746064,0.093979,NaN,-0.698870
apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,NaN,NaN,...,NaN,NaN,0.131203,0.149618,NaN,0.195211,0.127244,0.407013,NaN,0.801982
apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,NaN,NaN,...,NaN,NaN,0.273818,-0.328746,NaN,0.341081,0.284067,0.407942,NaN,0.297169
ofi_mth_cnt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ofi_mth_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


L ['부동산']


,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,ofi_mth_cnt,ofi_mth_price,ofi_yr_cnt,ofi_yr_price,total,L01,L03,L04
ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,NaN,NaN,NaN,NaN,0.787496,0.680087,NaN,NaN
ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,NaN,NaN,NaN,NaN,-0.752531,-0.638880,NaN,NaN
ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN
ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN
apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,NaN,NaN,NaN,NaN,0.372659,0.192039,NaN,NaN
apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,NaN,NaN,NaN,NaN,0.081457,0.399104,NaN,NaN
apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,NaN,NaN,NaN,NaN,0.306649,0.050607,NaN,NaN
apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,NaN,NaN,NaN,NaN,0.614235,0.469510,NaN,NaN
ofi_mth_cnt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ofi_mth_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


F ['생활서비스']


,ali_mth_cnt,ali_mth_price,ali_yr_cnt,ali_yr_price,apt_mth_cnt,apt_mth_price,apt_yr_cnt,apt_yr_price,ofi_mth_cnt,ofi_mth_price,...,F07,F08,F09,F11,F12,F13,F14,F15,F16,F20
ali_mth_cnt,1.000000,-0.998497,-1.0,1.0,0.903559,0.856464,0.830447,0.995610,NaN,NaN,...,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN
ali_mth_price,-0.998497,1.000000,-1.0,1.0,-0.878716,-0.883471,-0.798663,-0.999244,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN
ali_yr_cnt,-1.000000,-1.000000,1.0,-1.0,-1.000000,1.000000,-1.000000,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ali_yr_price,1.000000,1.000000,-1.0,1.0,1.000000,-1.000000,1.000000,-1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
apt_mth_cnt,0.903559,-0.878716,-1.0,1.0,1.000000,0.011590,0.763902,0.392595,NaN,NaN,...,-0.189506,NaN,NaN,0.346951,0.118467,-0.298323,-0.073163,NaN,-0.999943,NaN
apt_mth_price,0.856464,-0.883471,1.0,-1.0,0.011590,1.000000,-0.351454,0.390511,NaN,NaN,...,-0.383735,NaN,NaN,0.510238,-0.102126,-0.904461,-0.857685,NaN,0.633893,NaN
apt_yr_cnt,0.830447,-0.798663,-1.0,1.0,0.763902,-0.351454,1.000000,0.197144,NaN,NaN,...,-0.352592,NaN,NaN,-0.033548,0.278406,-0.098446,0.554435,NaN,-0.853837,NaN
apt_yr_price,0.995610,-0.999244,1.0,-1.0,0.392595,0.390511,0.197144,1.000000,NaN,NaN,...,0.892213,NaN,NaN,0.338988,0.544152,0.682399,0.146254,NaN,-0.117141,NaN
ofi_mth_cnt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ofi_mth_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
